In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn.metrics
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision as tv
from PIL import Image
from torchvision import models
import seaborn as sn
from torch.utils import data



In [ ]:
train_data = tv.datasets.ImageFolder('../input/skin-cancer-malignant-vs-benign/train', tv.transforms.ToTensor())
train_loader = data.DataLoader(dataset=train_data, batch_size=32, shuffle=True)

In [ ]:
test_data = tv.datasets.ImageFolder('../input/skin-cancer-malignant-vs-benign/test', tv.transforms.ToTensor())
test_loader = data.DataLoader(dataset=test_data, batch_size=32, shuffle=True)

In [ ]:
## RESNET50 ##
    
resnet = tv.models.resnet50(pretrained=True)
print(resnet)

In [ ]:
for param in resnet.parameters():
    param.requires_grad = False

resnet.fc = nn.Sequential(
    nn.Linear(200704, 1000),
    nn.Linear(1000, 10),
    nn.Linear(10,2))

print(resnet)

In [ ]:
resnet.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters())

total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.cuda(), labels.cuda()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

In [ ]:
def confm(model):
    conf_matrix = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            t_image, mask = data
            t_image, mask = (t_image.cuda()), (mask.cuda())
        
            output = model(t_image)
            pred = torch.argmax(output, 1)
            conf_matrix += sklearn.metrics.confusion_matrix(pred.cpu(), mask.cpu())
            
    df_cm = pd.DataFrame(conf_matrix, index = ['Malignant', 'Benign'], columns = ['Malignant', 'Benign'])
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='g') # font size

    plt.show()

In [ ]:
confm(resnet)

In [ ]:
## OUR MODEL ##

# Hyperparameters
num_epochs = 3
num_classes = 2
batch_size = 32
learning_rate = 0.001

class Net(torch.nn.Module):
    def __init__(self):
#         self.conv1 = nn.Conv2d(3, 10, 5)
#         self.conv2 = nn.Conv2d(10, 20, 5)

#         self.fc1 = nn.Linear(20 * 6 * 6, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 2)
        
        
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        
        self.fc1 = nn.Linear(200704, 1000)
        self.fc2 = nn.Linear(1000, 10)
        self.fc3 = nn.Linear(10,2)
        
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out
    
    

# def forward(x, self):
    
#     x = self.conv1(x)
# #    x = self.bn1(x)
#     x = torch.nn.functional.relu(x)
#     x = torch.nn.functional.max_pool2d(x, (2, 2))  
    
#     x = self.conv2(x)
# #    x = self.bn2(x)
#     x = torch.nn.functional.relu(x)
#     x = torch.nn.functional.max_pool2d(x, (2, 2))
    
#     ## FC layers ##
    
#     x = self.fc(x)
#     x = torch.nn.functional.sigmoid(x)
    
#     return x

In [ ]:
## Training The Model ##

model = Net()
model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

In [ ]:
confm(model)